In [1]:
import numpy as np
import pandas as pd
from utils import *
from lightgbm import LGBMClassifier
from feature_engineering import *
from check_score.scorer import Kaggle_score, get_model_score

# LGBM baseline

In [2]:
df_train = get_data_train()
df_test = get_data_test()

X_train = df_train.drop(columns=["Cover_Type"], axis=1)
y_train = df_train["Cover_Type"]

In [3]:
c_w ={1: 0.4, 2: 0.45, 3: 0.04,4: 0.01, 5: 0.04, 6: 0.04, 7: 0.04}

lgbm = LGBMClassifier(
    objective='multiclass',
    num_class=7,
    class_weight=c_w,
    boosting_type='gbdt',
    num_leaves=31,
    n_estimators=300,
    verbose=-1,
    n_jobs=-1
)

print(IWCV(df_train, lgbm, k_valid=10))

get_model_score(lgbm, df_train)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

## Soil Type

In [ ]:
def preprocess(df):
    df = concat_Soil_Type(df, new_name='ST_concat', drop_value=True) 
    df = concat_Wilderness_Area(df, new_name='WA_concat', drop_value=True) 
    df = group_climatic_zone(df)
    df = group_geological_zone(df)
    return df

df_train_new = preprocess(df_train)
cat_col = [ 'ST_concat', 'WA_concat','climatic_zone', 'geological_zone']

X_train = df_train_new.drop(columns=["Cover_Type"], axis=1)
y_train = df_train_new["Cover_Type"]
lgbm.fit(X_train, y_train, categorical_feature=cat_col)
pred = lgbm.predict(df_test)

pred = clean_predictor(y_pred=pred.Cover_Type, Id=pred.Id)
csv_for_submission(pred,"pred_test")
Kaggle_score('Output/pred_test')

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,ST_concat,geological_zone
0,242642,2881,130,22,210,54,1020,250,221,88,...,0,0,0,0,0,0,0,1,30,7
1,309891,3005,351,14,242,-16,1371,194,215,159,...,0,0,0,0,0,0,0,1,24,7
2,287847,3226,63,14,618,2,1092,232,210,107,...,0,0,0,0,0,0,0,1,29,7
3,516307,3298,317,8,661,60,752,198,233,174,...,0,0,0,0,0,0,0,1,23,2
4,124860,3080,35,6,175,26,3705,219,227,144,...,0,0,0,0,0,0,0,1,24,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,475155,3328,321,13,323,12,5109,186,227,180,...,0,0,0,0,1,0,0,7,38,7
15116,514378,3455,37,5,841,92,939,220,229,146,...,0,0,0,0,0,0,1,7,40,0
15117,368425,3279,90,14,404,113,1513,240,218,105,...,0,0,0,0,0,0,0,7,29,7
15118,537844,3589,357,9,418,52,1868,205,223,155,...,0,0,0,0,0,0,1,7,40,0
